In [ ]:
!/usr/local/bin/k3d cluster create kube-cluster \
  --agents 3 \
  --k3s-node-label topology.kubernetes.io/zone=zone-a@agent:0 \
  --k3s-node-label topology.kubernetes.io/zone=zone-b@agent:1 \
  --k3s-node-label topology.kubernetes.io/zone=zone-c@agent:2

In [ ]:
!kubectl apply -f sts.yaml

In [ ]:
! kubectl exec -it kafka-0 -- kafka-topics --create --topic test --partitions 3 --replication-factor 3 --bootstrap-server kafka-0.kafka-headless.kafka.svc.cluster.local:9092

In [ ]:
! kubectl exec -it kafka-0 -- kafka-topics --describe --topic test --bootstrap-server kafka-0.kafka-headless.kafka.svc.cluster.local:9092

In [ ]:
! kubectl get pods -n kafka -o wide